In [ ]:
import pandas as pd 
import seaborn as sns
import numpy as np
import  matplotlib.pyplot as plt

In [ ]:
# 导入数据
input_data = pd.read_excel(r'/Users/dylan/Github repo/DataBase/signal_process/benchmark_result-30_5.xlsx',sheet_name = 'Sheet1')
signal_input = pd.read_excel(r'/Users/dylan/Github repo/DataBase/signal_process/benchmark_result-30_5.xlsx',sheet_name = 'Sheet3')

signal_input_roll = signal_input[signal_input['model'] == 'benchmark']

In [ ]:
# 数据清理
signal_input_roll.drop(labels = 'Unnamed: 0',axis = 1, inplace = True)
signal_input_roll.reset_index(drop = True)
signal_input.drop(labels = 'Unnamed: 0',axis = 1, inplace = True)
signal_input.reset_index(drop = True)

In [ ]:
# 函数定义
def get_value(m):
    if 
    value = m['sharpe_ratio_sp'] - m[m['model'] == 'benchmark']['sharpe_ratio_sp'].iat[0] # 返回一个序列
    return value

def get_lable(n):
    if n >= 0:
        return True
    else: 
        return False


In [ ]:
# 进行无加权平均回溯
# 得到整个验证期的model_list
list_ = list(signal_input_roll.rolling(window = 10,min_periods = 10)['start_date'])[10:]    
model_list = pd.DataFrame()
list1 = []
for i in range(len(list_)):
    list1 = list1 + list(list_[i])
    x = signal_input.set_index('start_date',inplace = False).loc[list1]
    x['value'] = list(x.groupby('end_date').apply(get_value)) # 转为list
    x['lable'] = list(x['value'].apply(get_lable))
    x.loc[:,'value'] = x.transform({'value' : lambda x : np.square(x) if x >= 0 else (-np.square(x))})
    signal_input_value = x
    # input_data.groupby(['model','lable']).sum().apply(get_loss)
    input_filter = signal_input_value.groupby(['model','lable']).agg({'value' : 'sum'})
    input_filter.drop(input_filter.tail(1).index,inplace=True)
    input_filter = input_filter.unstack(fill_value = 0).loc[:,'value']
    input_filter.loc[:,'loss'] = input_filter[True] - input_filter[False]
    model_filter = input_filter.nlargest(10,'loss',keep = 'first')
    # model_list  = list(input_filter['loss'].nlargest(10).index.get_level_values(0).drop_duplicates())
    model_list = model_list.append(model_filter)

# 更改为链式写法



In [ ]:
# 统计频次
print(len(list1))
pd.Series(model_list.index).value_counts()

In [67]:
# 进行指数加权平均回溯

list_ = list(signal_input_roll.rolling(window = 10,min_periods = 10)['start_date'])[10:]    
model_list = pd.DataFrame()
list1 = []
input_filter_all = pd.DataFrame()
all_model = signal_input.model.drop_duplicates()
for i in range(len(list_)):
    list1 = list1 + list(list_[i])
    x = signal_input.set_index('start_date',inplace = False).loc[list1]
    x['value'] = list(x.groupby('end_date').apply(get_value)) # 转为list
    x['lable'] = list(x['value'].apply(get_lable))
    x.loc[:,'value'] = x.transform({'value' : lambda x : np.square(x) if x >= 0 else (-np.square(x))})
    # input_data.groupby(['model','lable']).sum().apply(get_loss)
    # 指数加权平均
    # input_filter = x.groupby(['model','lable']).emw(alpha = 0.9, adjust = False)
    input_filter = pd.DataFrame()
    for model in all_model:
        x[x.model == model].loc[:,'value'] = x[x.model == model].value.ewm(alpha = 0.6, adjust = False).mean()
        input_filter = input_filter.append(x[x.model == model])
    #input_filter.drop(input_filter.tail(1).index,inplace=True)
    #input_filter = input_filter.unstack(fill_value = 0).loc[:,'value']
    #input_filter.loc[:,'loss'] = input_filter[True] - input_filter[False]
    #input_filter_all = input_filter_all.append(input_filter)
    # model_list  = list(input_filter['loss'].nlargest(10).index.get_level_values(0).drop_duplicates())
    break

input_filter.head
		

/Users/dylan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


<bound method NDFrame.head of              end_date                model  annual_return_sp  sharpe_ratio_sp  \
start_date                                                                      
2018-05-09 2018-06-20  2日开盘价持续下跌-1120-bagc              0.12            -0.05   
2018-05-16 2018-06-27  2日开盘价持续下跌-1120-bagc             -4.59            -3.46   
2018-05-23 2018-07-04  2日开盘价持续下跌-1120-bagc             -7.67            -4.82   
2018-05-30 2018-07-11  2日开盘价持续下跌-1120-bagc             -2.54            -1.39   
2018-06-06 2018-07-18  2日开盘价持续下跌-1120-bagc             -2.01            -1.07   
...               ...                  ...               ...              ...   
2018-06-13 2018-07-25            benchmark             -2.63            -1.10   
2018-06-21 2018-08-01            benchmark             -1.64            -0.71   
2018-06-28 2018-08-08            benchmark              2.22             0.97   
2018-07-05 2018-08-15            benchmark              3.44             1.51  